In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine, inspect, text
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('online_gaming_behavior_dataset.csv')
df.head(5)

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.271119,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.531945,0,Medium,2,131,95,37,Medium


In [9]:
df.shape

(40034, 13)

In [4]:
path = "/sqldb.db"
# create db engine 
engine = create_engine(f'sqlite://{path}', echo=False)
# insert the csv data to sql table
#df.to_sql(name='online_gaming_behavior_dataset', con=engine)

In [5]:
# The Table name create in db
inspect(engine).get_table_names()

['online_gaming_behavior_dataset']

In [8]:

# Let's check if data is inserted correctly
conn = engine.connect()
res = conn.execute(text("SELECT * FROM online_gaming_behavior_dataset")).fetchall()
conn.close()
print(len(res))

40034


In [7]:
# we can also connect using the langchain library 
db = SQLDatabase.from_uri(f"sqlite://{path}")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM online_gaming_behavior_dataset LIMIT 10;")

sqlite
['online_gaming_behavior_dataset']


"[(0, 9000, 43, 'Male', 'Other', 'Strategy', 16.271118760553215, 0, 'Medium', 6, 108, 79, 25, 'Medium'), (1, 9001, 29, 'Female', 'USA', 'Strategy', 5.525961380570566, 0, 'Medium', 5, 144, 11, 10, 'Medium'), (2, 9002, 22, 'Female', 'USA', 'Sports', 8.223755243499511, 0, 'Easy', 16, 142, 35, 41, 'High'), (3, 9003, 35, 'Male', 'USA', 'Action', 5.265351277318268, 1, 'Easy', 9, 85, 57, 47, 'Medium'), (4, 9004, 33, 'Male', 'Europe', 'Action', 15.53194452113429, 0, 'Medium', 2, 131, 95, 37, 'Medium'), (5, 9005, 37, 'Male', 'Europe', 'RPG', 20.561855414112557, 0, 'Easy', 2, 81, 74, 22, 'Low'), (6, 9006, 25, 'Male', 'USA', 'Action', 9.752716365932256, 0, 'Hard', 1, 50, 13, 2, 'Low'), (7, 9007, 25, 'Female', 'Asia', 'RPG', 4.401729344841462, 0, 'Medium', 10, 48, 27, 23, 'Medium'), (8, 9008, 38, 'Female', 'Europe', 'Simulation', 18.15273259575482, 0, 'Easy', 5, 101, 23, 41, 'Medium'), (9, 9009, 38, 'Female', 'Other', 'Sports', 23.942771725289525, 0, 'Easy', 13, 95, 99, 36, 'High')]"

In [3]:
print("Are environment variables loaded?:", load_dotenv())
print("Is required API Key present?:", bool(os.getenv("GOOGLE_API_KEY")))

Are environment variables loaded?: True
Is required API Key present?: True


In [ ]:
# %pip install -qU langchain-google-genai
# To access Google AI models you'll need to create a Google Acount account, 
# get a Google AI API key, and install the langchain-google-genai integration package.
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=1000,
    timeout=None,
    max_retries=2,
)

In [ ]:
# Create sql agent using the langchain, it handles all required instrunction for sql agent.
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
# Let's ask question to Agent and get the answer from our dataset.
result = agent_executor.invoke({"input": "What is country with highest no. of players"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded:  Finally, I should construct a query to answer the question.

online_gaming_behavior_dataset
Invoking: `sql_db_schema` with `{'table_names': 'online_gaming_behavior_dataset'}`



CREATE TABLE online_gaming_behavior_dataset (
	"index" BIGINT, 
	"PlayerID" BIGINT, 
	"Age" BIGINT, 
	"Gender" TEXT, 
	"Location" TEXT, 
	"GameGenre" TEXT, 
	"PlayTimeHours" FLOAT, 
	"InGamePurchases" BIGINT, 
	"GameDifficulty" TEXT, 
	"SessionsPerWeek" BIGINT, 
	"AvgSessionDurationMinutes" BIGINT, 
	"PlayerLevel" BIGINT, 
	"AchievementsUnlocked" BIGINT, 
	"EngagementLevel" TEXT
)

/*
3 rows from online_gaming_behavior_dataset table:
index	PlayerID	Age	Gender	Location	GameGenre	PlayTimeHours	InGamePurchases	GameDifficulty	SessionsPerWeek	AvgSessionDurationMinutes	PlayerLevel	AchievementsUnlocked	EngagementLevel
0	9000	43	Male	Other	Strategy	16.271118760553215	0	Medium	6	108	79	25	Medium
1	9001	29	Female	USA	Strate

In [24]:
result

{'input': 'What is country with highest no. of players',
 'output': 'The country with the highest number of players is USA with 16000 players.'}

In [36]:
# below is the query run by Agent and same can be achieved by dataframe.
#query = SELECT Location, COUNT(*) AS PlayerCount FROM online_gaming_behavior_dataset GROUP BY Location ORDER BY PlayerCount DESC LIMIT 1;
df.groupby(by=['Location']).count().max()['PlayerID']

16000

In [ ]:
result = agent_executor.invoke({"input": "Do you think female players had played less compared to others and if so which country and age group has the minimum?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded:  Finally, I should construct a query to answer the question.

online_gaming_behavior_dataset
Invoking: `sql_db_schema` with `{'table_names': 'online_gaming_behavior_dataset'}`



CREATE TABLE online_gaming_behavior_dataset (
	"index" BIGINT, 
	"PlayerID" BIGINT, 
	"Age" BIGINT, 
	"Gender" TEXT, 
	"Location" TEXT, 
	"GameGenre" TEXT, 
	"PlayTimeHours" FLOAT, 
	"InGamePurchases" BIGINT, 
	"GameDifficulty" TEXT, 
	"SessionsPerWeek" BIGINT, 
	"AvgSessionDurationMinutes" BIGINT, 
	"PlayerLevel" BIGINT, 
	"AchievementsUnlocked" BIGINT, 
	"EngagementLevel" TEXT
)

/*
3 rows from online_gaming_behavior_dataset table:
index	PlayerID	Age	Gender	Location	GameGenre	PlayTimeHours	InGamePurchases	GameDifficulty	SessionsPerWeek	AvgSessionDurationMinutes	PlayerLevel	AchievementsUnlocked	EngagementLevel
0	9000	43	Male	Other	Strategy	16.271118760553215	0	Medium	6	108	79	25	Medium
1	9001	29	Female	USA	Strate

In [41]:
from IPython.display import display, Markdown
display(Markdown(result['input']))
display(Markdown(result['output']))

Do you think female players had played less compared to others and if which country and age group has the minimum?

The average playtime for females is 11.97 hours, while the average playtime for males is 12.06 hours. The location and age group with the minimum average playtime is 'Other' at age 40, with an average playtime of 10.64 hours.